In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch pandas nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import math

In [ ]:
dataset_path = "/content/drive/MyDrive/DSGP/template/bird_descriptions.txt"

# ✅ Read and Display First 5 Lines
with open(dataset_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines[:5]:  # Show first 5 lines
        print(line.strip())

Bird Name: Red-vented Bulbul
Scientific Name: Pycnonotus cafer
Description: A dark, sleek, medium-sized bird with a black crest and a white rump. The red color under the tail is often difficult to see. Eats fruit, flower buds, and insects. Conspicuous and sometimes gregarious, often seen high in trees or perched on wires in urban and rural areas; generally prefers scrubby edge habitat instead of dense forest. Calls include a variety of chirps and whistles. Native to South and Southeast Asia.
---
Bird Name: Blue-tailed Bee-eater


In [ ]:
# ✅ Load Pretrained GPT-2 Model & Tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset = load_dataset("text", data_files=dataset_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# ✅ Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BirdInfoModel/gpt2-bird-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,  # Increase to 5 or 6
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=1,
    logging_dir="/content/logs",
    report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# ✅ Data Collator (Handles Padding)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# ✅ Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<ipython-input-10-e12c3aa3c407>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,2.644877
2,No log,1.949207
3,No log,1.603821
4,No log,1.414847
5,No log,1.358487


TrainOutput(global_step=250, training_loss=2.163817138671875, metrics={'train_runtime': 6205.7338, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.04, 'total_flos': 130646016000000.0, 'train_loss': 2.163817138671875, 'epoch': 5.0})

In [ ]:
# ✅ Get Training Loss
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity (PPL): {math.exp(eval_results['eval_loss']):.4f}")

Validation Loss: 1.3585
Perplexity (PPL): 3.8903


In [ ]:
# ✅ Save Fine-Tuned Model
trainer.save_model("/content/drive/MyDrive/DSGP/BirdInfoModel/gpt2-bird-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/DSGP/BirdInfoModel/gpt2-bird-finetuned")

print("\n✅ Fine-Tuned GPT-2 Model Saved!")


✅ Fine-Tuned GPT-2 Model Saved!


In [ ]:
from transformers import pipeline

# ✅ Load Fine-Tuned Model
model_path = "/content/drive/MyDrive/DSGP/BirdInfoModel/gpt2-bird-finetuned"
gpt2_pipe = pipeline("text-generation", model=model_path, tokenizer=model_path)

# ✅ Generate Sample Description
ai_description = gpt2_pipe(
    "The White-throated Kingfisher is a bird that",
    max_new_tokens=100,
    num_return_sequences=1
)[0]["generated_text"]

print("\n🔹 AI-Generated Bird Description:\n", ai_description)


Device set to use cpu



🔹 AI-Generated Bird Description:
 The White-throated Kingfisher is a bird that is frequently found in urban water bodies around Australia, where it is known for its call and calls are a wide variety of languages. Black-headed kingfisher, common in the coastal areas of Australia, North Africa and Asia; commonly seen out swimming in open water due to its varied patterns of feeding and grooming. Commonly found near wetlands, including riverside gardens, catchments and agricultural zones, often seen foraging on the shore. Commonly found in dense dense forests, plantations, and urban
